In [1]:
import pandas as pd
import psycopg2

In [2]:
hv_map = pd.read_csv('numeric_prio_fields.csv')

In [3]:
db_connection = psycopg2.connect(user='postgres',
                                 password='insertyourpassword',
                                 host='localhost',
                                 port='5432',
                                 database='UK_BB')

In [4]:
query_concepts = "select concept_id_2 \
                    from vocab.concept join vocab.concept_relationship on concept_id = concept_id_1 \
                    where vocabulary_id = 'UK Biobank' and concept_code = '%(code)s' \
                    and relationship_id = 'Maps to'"
query_units = "select concept_id_2 \
                from vocab.concept join vocab.concept_relationship on concept_id = concept_id_1 \
                where vocabulary_id = 'UK Biobank' and concept_code = '%(code)s' \
                and relationship_id = 'Maps to unit'"

In [5]:
codes = hv_map['sourceCode'].unique().tolist()

In [6]:
cursor = db_connection.cursor()

all_possible = []
for c in codes:
    field_id = c
    field_name = hv_map[hv_map['sourceCode']==c]['sourceName'].unique()[0]
    cursor.execute(query_concepts, {'code': c})
    a8_concept = cursor.fetchone()
    cursor.execute(query_units, {'code': c})
    a8_unit = cursor.fetchone()
    hv_concept = hv_map[(hv_map['sourceCode']==c) & (hv_map['mappingType']=='EVENT')]['conceptId'].tolist()
    hv_unit = hv_map[(hv_map['sourceCode']==c) & (hv_map['mappingType']=='UNIT')]['conceptId'].tolist()
    if a8_concept is None:
        a8_concept = (0,)
    if a8_unit is None:
        a8_unit = (0,)
    if not hv_concept:
        hv_concept = [0]
    if not hv_unit:
        hv_unit = [0]
    all_possible.append([field_id, field_name, a8_concept[0], a8_unit[0], hv_concept[0], hv_unit[0]])
print(all_possible)

[[30850, 'Testosterone', 3017446, 8736, 4020107, 8736], [30720, 'Cystatin C', 3030366, 8751, 44789161, 8748], [30250, 'Reticulocyte count', 40763528, 44777575, 4263581, 4079010], [2744, 'Birth weight of first child', 35810314, 8739, 0, 8739], [30150, 'Eosinophill count', 3028615, 44777588, 4216098, 4175206], [30050, 'Mean corpuscular haemoglobin', 3012030, 8564, 4182871, 8564], [30650, 'Aspartate aminotransferase', 3013721, 8645, 4263457, 8645], [30324, 'Volume of EDTA1 red cells held by UKB', 4105828, 9665, 0, 9665], [30860, 'Total protein', 3020630, 8636, 4119650, 8636], [2754, 'Age at first live birth', 0, 9448, 0, 9448], [30011, 'Red blood cell (erythrocyte) count freeze-thaw cycles', 0, 0, 4030871, 0], [30111, 'Platelet distribution width freeze-thaw cycles', 0, 0, 4097620, 0], [30730, 'Gamma glutamyltransferase', 3026910, 8645, 4289475, 8645], [30251, 'Reticulocyte count freeze-thaw cycles', 0, 0, 4263581, 0], [3546, 'Age last used hormone-replacement therapy (HRT)', 35810380, 94

In [7]:
df_results = pd.DataFrame(all_possible)
df_results.columns = ['field_id', 'field_name', 'a8_concept', 'a8_unit', 'hv_concept', 'hv_unit']
df_results['concept_match'] = (df_results['a8_concept'] == df_results['hv_concept'])
df_results['unit_match'] = (df_results['a8_unit'] == df_results['hv_unit'])


In [8]:
query_details = "select domain_id, vocabulary_id from vocab.concept where concept_id = '%(code2)s'"

In [9]:
a8_concept_dtl = []
a8_unit_dtl = []
hv_concept_dtl = []
hv_unit_dtl = []
n = df_results.shape[0]
for i in range(n):
    cursor.execute(query_details, {'code2': df_results['a8_concept'].tolist()[i]})
    a8_c = cursor.fetchone()
    a8_concept_dtl.append(a8_c)
    cursor.execute(query_details, {'code2': df_results['a8_unit'].tolist()[i]})
    a8_u = cursor.fetchone()
    a8_unit_dtl.append(a8_u)
    cursor.execute(query_details, {'code2': df_results['hv_concept'].tolist()[i]})
    hv_c = cursor.fetchone()
    hv_concept_dtl.append(hv_c)
    cursor.execute(query_details, {'code2': df_results['hv_unit'].tolist()[i]})
    hv_u = cursor.fetchone()
    hv_unit_dtl.append(hv_u)

In [10]:
df_results.insert(loc=3, column='a8_concept_details', value=a8_concept_dtl)
df_results.insert(loc=5, column='a8_unit_details', value=a8_unit_dtl)
df_results.insert(loc=7, column='hv_concept_details', value=hv_concept_dtl)
df_results.insert(loc=9, column='hv_unit_details', value=hv_unit_dtl)

In [11]:
pd.set_option('display.max_rows', 500)
df_results[df_results['concept_match'] == False] #.count()

,field_id,field_name,a8_concept,a8_concept_details,a8_unit,a8_unit_details,hv_concept,hv_concept_details,hv_unit,hv_unit_details,concept_match,unit_match
0,30850,Testosterone,3017446,"(Measurement, LOINC)",8736,"(Unit, UCUM)",4020107,"(Measurement, SNOMED)",8736,"(Unit, UCUM)",False,True
1,30720,Cystatin C,3030366,"(Measurement, LOINC)",8751,"(Unit, UCUM)",44789161,"(Measurement, SNOMED)",8748,"(Unit, UCUM)",False,False
2,30250,Reticulocyte count,40763528,"(Measurement, LOINC)",44777575,"(Unit, UCUM)",4263581,"(Measurement, SNOMED)",4079010,"(Observation, SNOMED)",False,False
3,2744,Birth weight of first child,35810314,"(Observation, UK Biobank)",8739,"(Unit, UCUM)",0,"(Metadata, None)",8739,"(Unit, UCUM)",False,True
4,30150,Eosinophill count,3028615,"(Measurement, LOINC)",44777588,"(Unit, UCUM)",4216098,"(Measurement, SNOMED)",4175206,"(Observation, SNOMED)",False,False
5,30050,Mean corpuscular haemoglobin,3012030,"(Measurement, LOINC)",8564,"(Unit, UCUM)",4182871,"(Measurement, SNOMED)",8564,"(Unit, UCUM)",False,True
6,30650,Aspartate aminotransferase,3013721,"(Measurement, LOINC)",8645,"(Unit, UCUM)",4263457,"(Measurement, SNOMED)",8645,"(Unit, UCUM)",False,True
7,30324,Volume of EDTA1 red cells held by UKB,4105828,"(Specimen, SNOMED)",9665,"(Unit, UCUM)",0,"(Metadata, None)",9665,"(Unit, UCUM)",False,True
8,30860,Total protein,3020630,"(Measurement, LOINC)",8636,"(Unit, UCUM)",4119650,"(Measurement, SNOMED)",8636,"(Unit, UCUM)",False,True
10,30011,Red blood cell (erythrocyte) count freeze-thaw...,0,"(Metadata, None)",0,"(Metadata, None)",4030871,"(Measurement, SNOMED)",0,"(Metadata, None)",False,True


In [12]:
df_results.to_csv('all_concepts_numeric_prio.csv', sep=',', index=False)